In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="d9553335-9582-4258-9b03-a23a4b1c4b20")

In [4]:
index_name = "medicalchatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf(r"C:\Users\91883\chatbot\data")

In [8]:
#extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\91883\AppData\Local\Temp\ipykernel_14208\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\91883\chatbot\env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
#query_result

In [16]:
pc.describe_index('medicalchatbot')


{'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'medicalchatbot-q8h8v1k.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'medicalchatbot',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [17]:
# Connect to the Pinecone index using the host parameter
#index = pinecone.Index(index_name=index_name, host="medical-chatbot-q8h8v1k.svc.aped-4627-b74a.pinecone.io")

In [18]:
index = pinecone.Index("medical-chatbot","medicalchatbot-q8h8v1k.svc.aped-4627-b74a.pinecone.io")

In [22]:
# Initialize the model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [21]:
import numpy as np

# Create embeddings for each text chunk
embeddings = []
for chunk in text_chunks:
    text = chunk.page_content
    embedding = model.encode(text)  # Use the encode method to generate embeddings
    embeddings.append(embedding.tolist())  # Convert the NumPy array to a list

# Upsert the embeddings into the index
index.upsert(vectors=embeddings, ids=[f"chunk-{i}" for i in range(len(text_chunks))])

ValueError: Invalid vector value passed: cannot interpret type <class 'list'>

In [17]:
# Create embeddings for each chunk and upsert to Pinecone
ids = [f"chunk_{i}" for i in range(len(text_chunks))]
vectors = [(ids[i], embeddings.embed_query(text_chunks[i].page_content)) for i in range(len(text_chunks))]

In [18]:
# Upsert the embeddings into the Pinecone index
index.upsert(vectors=vectors)

print("Embeddings successfully stored in Pinecone!")

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 02 Oct 2024 11:56:25 GMT', 'Content-Type': 'text/plain', 'Content-Length': '9', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Wrong API key', 'www-authenticate': 'Wrong API key', 'server': 'envoy'})
HTTP response body: Forbidden


In [ ]:
#Creating Embeddings for Each of The Text Chunks & storing
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])